In [1]:
#libraries
import numpy as np 
from math import cos
import pandas as pd

**Вариант 2. Задание 11.**

\begin{equation*}
 \begin{cases}
    \frac{\partial u}{\partial t} = \frac{\partial}{r^2 \partial r}(r^2u\frac{\partial u}{\partial r}) + \frac{\partial}{r^2sin\theta \partial \theta}(sin\theta u\frac{\partial u}{\partial \varphi}) , 0 < t,r \le 1, 0 < \varphi < \pi / 2 \\
    u(0, r, \theta) = \frac{r^2cos^2(\theta)}{7}, 0 \le r \le 1, 0 \le \theta \le \pi / 2 \\
    u(t. 0, \theta) = 0, 0 < t \le 1, 0 \le \varphi \le \pi / 2\\
    u(t, 1, \theta) = \frac{cos^2(\theta)}{(7 - 6t)}, 0 < t \le 1, 0 \le \theta < \pi / 2 \\
    u(t, r, 0) = \frac{r^2}{(7 - 6t)}, 0 < r < 1, 0 < t \le 1 \\
    u(t, r, \pi / 2) = 0, 0 < r < 1, 0 < t \le 1    
 \end{cases}
\end{equation*}

In [2]:
def f_private(t, r, phi):
    '''
    функция частного решения
    '''
    return r**2 * (np.cos(phi))**2 / (7 - 6 * t)

def private_solution(t, r, phi):
    '''
    частное решение на сетке
    '''
    u = np.zeros((len(phi), len(r)))
    for i in range(len(phi)):
        u[i, :] = f_private(t, r, phi[i])
    return u

In [3]:
def grid(L, M, N):
    '''
    returns np.arrays of x, t broken into L and N pieces
    '''
    return np.linspace(0, 1, L), np.linspace(0, np.pi /2, M), np.linspace(0, 1, N)#менять первый и второй параметр, если изменятся границы

def sweep_mthd(a, b, c, d, m, l, u_ex):
    '''
    Метод прогонки
    '''
    alpha = np.zeros((m, l))
    beta = np.zeros((m, l))
    # beta[1:-1, 0] = u_ex[1:-1,0]
    
    x = u_ex.copy()
    
    for i in range(1, l - 1):
        alpha[1:-1,i] = (-a(i) / (c(i) * alpha[1:-1,i-1] + b(i)))
        beta[1:-1,i] = (d(i) - c(i) * beta[1:-1,i-1]) / (b(i) + c(i) * alpha[1:-1,i-1])
    
    for i in reversed(range(1, l-1)):
        x[1:-1,i] = alpha[1:-1,i] * x[1:-1,i + 1]  + beta[1:-1,i]
    return x

def accuracy(u1, u2):
    '''
    Вычислят точность
    Returns bool
    '''
    max = 0
    epsilon = 0.01#точность
    for m in range(u1.shape[1]):
        for l in range(u1.shape[0]):
            if u2[m][l] != 0:
                num = abs((abs(u2[m][l]) - abs(u1[m][l])) / u2[m][l])
                if num - max > 0:
                    max = num
            if max - epsilon > 0:
                return False
    return True

def first_step(u, u1, r, m, n, mu, tau, hr):
    
    def a_l(l):
        return -(hr * (l + 0.5))**2 * (u1[1:-1, l + 1]**mu + u1[1:-1, l]**mu) * tau / (2 * (r[l] * hr)**2)
    def c_l(l):
        return -(hr * (l - 0.5))**2 * (u1[1:-1, l]**mu + u1[1:-1, l - 1]**mu) * tau / (2 * (r[l] * hr)**2)
    def b_l(l):    
        return 1 - a_l(l) - c_l(l)
    def d_l(l):
        return u[n, 1:- 1, l]
    l = len(r)
    u_p = sweep_mthd(a_l, b_l, c_l, d_l, m, l, u[n+1,:,:])

    return u_p

def second_step(u, u_tilda, u_f, r, m, mu, tau, hf):
    l = len(r)
    u1 = u
    u_tilda1 = u_tilda.T

    def a_m(m):
        return -cos((m + 0.5) * hf) * (u1[1:-1, m + 1]**mu + u1[1:-1, m]**mu) * tau / (2 * cos(m * hf) * (r[1:-1]* hf)**2)
    def c_m(m):
        return -cos((m - 0.5) * hf) *(u1[1:-1, m]**mu + u1[1:-1, m - 1]**mu) * tau / (2 * cos(m * hf) * (r[1:-1] * hf)**2)
    def b_m(m):
        return 1 - a_m(m) - c_m(m)
    def d_m(m):
        return u_tilda1[1:-1, m]
    
    return sweep_mthd(a_m, b_m, c_m, d_m, l, m, u_f.T).T

    
    
def numerical_solve(r, phi, t, mu):
    hr = 1 / (len(r) - 1)
    hf = np.pi / 2 / (len(phi) - 1)
    tau = 1 / (len(t) - 1)
    
    m = len(phi)
    
    u = np.zeros((len(t), len(phi), len(r)))
    
    for i in range(len(phi)):#первое граничное условие
        u[0, i, :] = f_private(0, r, phi[i])
    
    for i in range(1, len(t)):
        u[i, :, -1] = f_private(t[i], r[-1], phi)#третье граничное условие
        u[i, 0, 1:-1] = f_private(t[i], r[1:-1], phi[0])#четвертое граничное условие
   
    for i in range(len(t) - 1):
        u_smth = u[i].copy()
        
        while True:
            u_tilda = first_step(u, u_smth, r, m, i, mu, tau, hr)
            u_end = second_step(u_smth, u_tilda, u[i+1], r, m, mu, tau, hf)
            if accuracy(u_smth, u_end):
                u[i + 1] = u_end
                break
            else:
                u_smth = u_end
             
    return u

In [4]:
#ТО ШО МЕНЯТЬ
L = M = 6
N = 6
mu = 1
T = 1

#находим аналитическое решение
r_rep, phi_rep, _ = grid(6, 6, 6) 

u_private = private_solution(T, r_rep, phi_rep)


#находим численное рашение
r, phi, t = grid(L, M, N)
u_num = numerical_solve(r, phi, t, mu)

if L != 6:
    u_numeric = u_num[-1,::2, ::2]#L=21 двойки меняем на 4, L=41 меняем 4 на 8 и так далее
else:
    u_numeric = u_num[-1]

print('Max error:', np.max(np.abs(u_private - u_numeric))/2)



Max error: 0.07537035173877116


In [5]:
print('_____Analytical____')
f = pd.DataFrame(u_private, columns=r_rep, index=phi_rep)
f

_____Analytical____


,0.0,0.2,0.4,0.6,0.8,1.0
0.000000,0.0,4.000000e-02,1.600000e-01,3.600000e-01,6.400000e-01,1.000000e+00
0.314159,0.0,3.618034e-02,1.447214e-01,3.256231e-01,5.788854e-01,9.045085e-01
0.628319,0.0,2.618034e-02,1.047214e-01,2.356231e-01,4.188854e-01,6.545085e-01
0.942478,0.0,1.381966e-02,5.527864e-02,1.243769e-01,2.211146e-01,3.454915e-01
1.256637,0.0,3.819660e-03,1.527864e-02,3.437694e-02,6.111456e-02,9.549150e-02
1.570796,0.0,1.499760e-34,5.999039e-34,1.349784e-33,2.399616e-33,3.749399e-33


In [6]:
print('_____Numerical____')
f1 = pd.DataFrame(u_numeric, columns=r_rep, index=phi_rep)
f1


_____Numerical____


,0.0,0.2,0.4,0.6,0.8,1.0
0.000000,0.0,0.040000,0.160000,0.360000,0.640000,1.000000e+00
0.314159,0.0,0.072122,0.264341,0.476364,0.681305,9.045085e-01
0.628319,0.0,0.059872,0.206243,0.350242,0.492739,6.545085e-01
0.942478,0.0,0.045857,0.147265,0.222002,0.281826,3.454915e-01
1.256637,0.0,0.030986,0.094946,0.126637,0.123857,9.549150e-02
1.570796,0.0,0.000000,0.000000,0.000000,0.000000,3.749399e-33


In [7]:
print('_____Errors____')
fe =  pd.DataFrame(np.abs(u_private - u_numeric)/2, columns=r_rep, index=phi_rep)
fe

_____Errors____


,0.0,0.2,0.4,0.6,0.8,1.0
0.000000,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
0.314159,0.0,1.797096e-02,5.980975e-02,7.537035e-02,5.120992e-02,0.0
0.628319,0.0,1.684563e-02,5.076100e-02,5.730924e-02,3.692663e-02,0.0
0.942478,0.0,1.601847e-02,4.599321e-02,4.881259e-02,3.035572e-02,0.0
1.256637,0.0,1.358315e-02,3.983378e-02,4.613027e-02,3.137109e-02,0.0
1.570796,0.0,7.498799e-35,2.999520e-34,6.748919e-34,1.199808e-33,0.0
